# SKIP

In [ ]:
class SKIPKernel(Kernel):
    """
    Scalable Kernel Interpolation for Product Kernels (SKIP) using Matern Kernels.
    """

    def __init__(self, base_kernel_class, grid_size=40, num_dims=1, grid_bounds=None, nu=1.5, **kwargs):
        """
        Initializes the SKIP kernel.

        Args:
            base_kernel_class (class): The base Matern kernel class (e.g., MaternKernel).
            grid_size (int): Number of grid points per dimension.
            num_dims (int): Number of input dimensions.
            grid_bounds (list of tuples, optional): Bounds for each dimension.
            nu (float): Smoothness parameter for the Matern kernel.
        """
        super(SKIPKernel, self).__init__(**kwargs)
        self.num_dims = num_dims

        # Initialize a GridInterpolationKernel for each dimension using torch.nn.ModuleList
        self.base_kernels = ModuleList([
            GridInterpolationKernel(
                base_kernel_class(nu=nu, ard_num_dims=1),
                grid_size=grid_size,
                num_dims=1,
                grid_bounds=[grid_bounds[d]] if grid_bounds is not None else None,
                active_dims=[d]  # Specify the active dimension
            )
            for d in range(num_dims)
        ])

        # Combine the univariate kernels using ProductKernel
        self.product_kernel = ProductKernel(*self.base_kernels)

        # Apply scaling to the combined kernel
        self.scale_kernel = ScaleKernel(self.product_kernel)

    def forward(self, x1, x2, **params):
        """
        Computes the covariance matrix between x1 and x2.

        Args:
            x1 (torch.Tensor): First input tensor of shape [n1, d].
            x2 (torch.Tensor): Second input tensor of shape [n2, d].

        Returns:
            gpytorch.lazy.LazyTensor: The covariance matrix.
        """
        cov = self.scale_kernel(x1, x2, **params)
        return cov

# Modify the GPRegressionModel class to use SKIP with Matern kernels
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, grid_size=40, grid_bounds=None, nu=0.5):
        """
        Initializes the GP Regression Model using SKIP with Matern Kernels.

        Args:
            train_x (torch.Tensor): Training inputs of shape [num_samples, D].
            train_y (torch.Tensor): Training targets of shape [num_samples].
            likelihood (gpytorch.likelihoods.GaussianLikelihood): Likelihood for the GP.
            grid_size (int): Number of grid points per dimension for SKIP.
            grid_bounds (list of tuples, optional): Bounds for each dimension.
            nu (float): Smoothness parameter for the Matern kernel.
        """
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        # Ensure grid_bounds are set correctly; if not provided, assume [0, 1] for each dimension
        if grid_bounds is None:
            grid_bounds = [(0.0, 1.0) for _ in range(train_x.shape[-1])]

        self.mean_module = ConstantMean()

        # Initialize the SKIP kernel with Matern base kernels
        self.covar_module = SKIPKernel(
            base_kernel_class=MaternKernel,
            grid_size=grid_size,
            num_dims=train_x.shape[-1],
            grid_bounds=grid_bounds,
            nu=nu
        )

    def forward(self, x):
        """
        Computes the Multivariate Normal distribution for input x.

        Args:
            x (torch.Tensor): Input tensor of shape [n, D].

        Returns:
            gpytorch.distributions.MultivariateNormal: The resulting distribution.
        """
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x, x)
        return MultivariateNormal(mean_x, covar_x)

def train_gp_model(train_x, train_y, patience=200, min_delta=1e-6, max_iterations=500, grid_size=100, nu=0.5):
    """
    Trains a Gaussian Process Regression model with SKIP and early stopping.

    Args:
        train_x (torch.Tensor): Training inputs of shape [num_samples, D].
        train_y (torch.Tensor): Training targets of shape [num_samples].
        patience (int): Number of iterations to wait for improvement before stopping.
        min_delta (float): Minimum change in the loss to qualify as an improvement.
        max_iterations (int): Maximum number of iterations to run.
        grid_size (int): Number of grid points per dimension for SKIP.
        grid_bounds (list of tuples, optional): Bounds for each dimension.
        nu (float): Smoothness parameter for the Matern kernel.

    Returns:
        model (GPRegressionModelSKIP): Trained GP model.
        likelihood (gpytorch.likelihoods.GaussianLikelihood): Associated likelihood.
    """
    # Example normalization
    train_x = torch.clamp(train_x, min=0.0, max=1.0)    
    # Initialize the likelihood and model
    likelihood = gpytorch.likelihoods.GaussianLikelihood(
        noise_constraint=gpytorch.constraints.GreaterThan(1e-7)
    )
    
    # Ensure grid_bounds are set to [0, 1] if not provided
    # grid_bounds = [(-0.005, 1.005) for _ in range(train_x.shape[-1])]
    grid_bounds = [(0.0, 1.0) for _ in range(train_x.shape[-1])]
    
    model = GPRegressionModel(
        train_x=train_x,
        train_y=train_y,
        likelihood=likelihood,
        grid_size=grid_size,
        grid_bounds=grid_bounds,
        nu=nu
    )
    model.train()
    likelihood.train()

    # Use the Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

    # Define the Marginal Log Likelihood (MLL)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # Variables for early stopping
    best_loss = float('inf')
    no_improvement_count = 0

    for i in range(max_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

        current_loss = loss.item()

        # Check for improvement
        if current_loss < best_loss - min_delta:
            best_loss = current_loss
            no_improvement_count = 0  # Reset counter
        else:
            no_improvement_count += 1  # Increment counter

        # Early stopping condition
        if no_improvement_count >= patience:
            print(f"Early stopping at iteration {i+1}")
            break

    return model, likelihood


===== Portfolio Optimization Parameters =====
Number of Assets (D): 2
Total Years (T): 6
Number of Time Steps (M): 12
Time Step Size (Delta_t): 0.5
Discount Factor (beta): 0.951229424500714
Relative Risk Aversion (gamma): 2.5
Transaction Cost Rate (tau): 0.005
Yearly Net Risk-Free Rate (r): 0.04
Expected Yearly Net Returns (mu): [0.06 0.06]
Covariance Matrix (Sigma):
[[0.04 0.  ]
 [0.   0.04]]
Include Consumption: False
Minimum Consumption (c_min): 0.0
Number of State Points (N): 100
merton_p: [0.2 0.2]
Integration Method: quadrature

Time step 11
include consumption: False
Step 2a: Approximate NTR


/opt/anaconda3/envs/Peytz2/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
/opt/anaconda3/e

[[0.1758 0.1758]
 [0.1753 0.2166]
 [0.1751 0.2163]
 [0.1749 0.216 ]
 [0.1747 0.2158]
 [0.2166 0.1753]
 [0.2157 0.2157]
 [0.2163 0.1751]
 [0.2157 0.2157]
 [0.2155 0.2155]
 [0.216  0.1749]
 [0.2158 0.1747]]
len tilde_omega_t: 12
Step 2b: Sample state points
Best solution found. Point tensor([0.1250, 0.8750]), Delta+: [0.0499 0.    ], Delta-: [0.    0.659], Delta: [ 0.0499 -0.659 ], Omega: [[0.1749 0.216 ]], bt: 0.6055
Best solution found. Point tensor([0.2083, 0.7917]), Delta+: [0. 0.], Delta-: [0.     0.5761], Delta: [ 0.     -0.5761], Omega: [[0.2083 0.2156]], bt: 0.5732
Best solution found. Point tensor([0.0833, 0.9167]), Delta+: [0.0915 0.    ], Delta-: [0.     0.7007], Delta: [ 0.0915 -0.7007], Omega: [[0.1748 0.216 ]], bt: 0.6053
Best solution found. Point tensor([0.1667, 0.8333]), Delta+: [0.0083 0.    ], Delta-: [0.     0.6172], Delta: [ 0.0083 -0.6172], Omega: [[0.175  0.2161]], bt: 0.6058
Best solution found. Point tensor([0.0417, 0.4167]), Delta+: [0.1335 0.    ], Delta-: [0. 

KeyboardInterrupt: 